# 졸업 논문

## DSAN: Denoising Self-Attention Network

- 추가 실험 jupyter 코드

### colab setting

In [ ]:
# import os
# from google.colab import drive

# drive.mount('/drive/')

# experiment_dir = '/drive/MyDrive/experiments'
# os.chdir(experiment_dir)

In [1]:
!nvidia-smi

zsh:1: command not found: nvidia-smi


In [5]:
!pip install missingpy

  Using cached missingpy-0.2.0-py3-none-any.whl (49 kB)


In [6]:
import sys
import numpy as np
import pandas as pd

from missingpy import MissForest
from dsan import Imputer as DSANImputer
from utils import *

import warnings
warnings.filterwarnings('ignore')

In [3]:
seed = 128
set_seed(seed)

In [17]:
def get_imputer(config):
    
    model = config['model']
    
    if model == 'DSAN':
        imputer = DSANImputer(rep_dim=config['rep_dim'],
                num_heads=config['num_heads'],
                n_hidden=config['n_hidden'],
                lr=config['lr'],
                weight_decay=config['weight_decay'],
                batch_size=config['batch_size'],
                epochs=config['epochs'],
                noise_percent=config['noise_percent'],
                stopped_epoch=config['stopped_epoch'])
    
    elif model == 'MissForest':
        imputer = MissForest(random_state=0)
        
    elif model == 'Statistics':
        imputer = None
        
    return imputer

In [19]:
def experiment_run(data_name, config, test=False):
    
    dataset = get_data(data_name)
    if test:
        dataset = (dataset[0][:1000], dataset[1], dataset[2], dataset[3])

    k = 5
    percent_missing_lst = [ p for p in range(5, 25, 5)]    
    X_origin, n_col, num_vars, cat_vars = dataset
    X_origin = category_mapping(X_origin, cat_vars)
    
    print("Data Size: {}".format(X_origin.shape))

    n_sample = X_origin.shape[0]
    idx_lst = list(range(n_sample))
    np.random.shuffle(idx_lst)

    # k-fold validation
    n_valid = len(idx_lst) // k
   
    results = {}
    for percent_missing in percent_missing_lst:
        result = []
        for it, n in enumerate(range(0, n_sample, n_valid)):
            valid_idx = idx_lst[n: n+n_valid]

            if len(valid_idx) < n_valid:
                break

            train_idx = list(set(idx_lst) - set(valid_idx))
            
            X = X_origin[train_idx, :]
            X_test = X_origin[valid_idx, :] # for prediction aucroc metric
            
            # make incomplete data
            missing_mask = generate_missing_mask(X, percent_missing=percent_missing)
            X_incomplete = X.copy()
            X_incomplete[missing_mask] = np.nan

            imputer = get_imputer(config)
            X_imputed = imputer.fit_transform(X_incomplete, cat_vars=cat_vars)
            
            X_res = X.copy()
            X_res[missing_mask] = X_imputed[missing_mask]
            
            metric = cal_metric_numpy(X_res, X, missing_mask, num_vars, cat_vars)
            score = ex_classify(data_name=data_name,
                    train_array=X_res,
                    test_array=X_test,
                    num_vars=num_vars,
                    cat_vars=cat_vars)
            
            metric['clf_aucroc'] = round(score, 4)
            result.append(metric)
            print(metric)
    
        avg_result = dict()
        for key in result[0].keys():
            value = np.mean([m[key] for m in result])
            avg_result[key] = value 
            print(key, value, percent_missing)

        results[percent_missing] = avg_result

    return results

In [26]:
config = {
    'model': 'DSAN',
    'rep_dim': 32,
    'num_heads': 8,
    'n_hidden': 2,
    'lr': 3e-3,
    'weight_decay': 1e-5,
    'batch_size': 128,
    'epochs': 10,
    'noise_percent': 10,
    'stopped_epoch': 10
}

In [40]:
import missingpy

In [43]:
from missingpy import MissForest

In [38]:
data_name_lst = ['adult', 'bank', 'crime']

result_dir = './results'
os.makedirs(result_dir)

for data_name in data_name_lst:
    results = experiment_run(data_name, config, test=True)
    results['config'] = config
    results['dataset'] = data_name
    result_path = os.path.join(result_dir, '{}.json'.format(data_name))
    write_json(results, result_path)

  0%|          | 0/10 [00:00<?, ?it/s]

Data Size: (1000, 15)


  0%|          | 0/10 [00:00<?, ?it/s]

{'nrmse': 0.4611, 'col_1_error_rate': 0.3659, 'col_3_error_rate': 0.2778, 'col_5_error_rate': 0.16, 'col_6_error_rate': 0.6857, 'col_7_error_rate': 0.1633, 'col_8_error_rate': 0.1622, 'col_9_error_rate': 0.1556, 'col_13_error_rate': 0.1304, 'col_14_error_rate': 0.15, 'total_error_rate': 0.2375, 'clf_aucroc': 0.8678}


  0%|          | 0/10 [00:00<?, ?it/s]

...

{'nrmse': 0.6205, 'col_1_error_rate': 0.3924, 'col_3_error_rate': 0.303, 'col_5_error_rate': 0.2516, 'col_6_error_rate': 0.8194, 'col_7_error_rate': 0.3758, 'col_8_error_rate': 0.0838, 'col_9_error_rate': 0.1366, 'col_13_error_rate': 0.0839, 'col_14_error_rate': 0.1908, 'total_error_rate': 0.2918, 'clf_aucroc': 0.8684}


  0%|          | 0/10 [00:00<?, ?it/s]

{'nrmse': 0.5677, 'col_1_error_rate': 0.3913, 'col_3_error_rate': 0.3476, 'col_5_error_rate': 0.2603, 'col_6_error_rate': 0.7748, 'col_7_error_rate': 0.2857, 'col_8_error_rate': 0.0889, 'col_9_error_rate': 0.1776, 'col_13_error_rate': 0.0651, 'col_14_error_rate': 0.1325, 'total_error_rate': 0.2736, 'clf_aucroc': 0.9103}
nrmse 0.5241800000000001 20
col_1_error_rate 0.39360000000000006 20
col_3_error_rate 0.31189999999999996 20
col_5_error_rate 0.23403999999999997 20
col_6_error_rate 0.7420599999999999 20
col_7_error_rate 0.27586 20
col_8_error_rate 0.08641999999999998 20
col_9_error_rate 0.14352 20
col_13_error_rate 0.06416000000000001 20
col_14_error_rate 0.16709999999999997 20
total_error_rate 0.26648 20
clf_aucroc 0.8730399999999999 20
Data Size: (1000, 17)


100%|██████████| 10/10 [00:06<00:00,  1.44it/s]


ValueError: Only one class present in y_true. ROC AUC score is not defined in that case.

In [23]:
config = {
    'model': 'MissForest'
    }

In [24]:
experiment_run('crime', config, test=True)

b'Skipping line 1513591: expected 23 fields, saw 24\n'


Data Size: (1000, 12)
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iteration: 9
{'nrmse': 0.0058, 'col_0_error_rate': 0.175, 'col_1_error_rate': 0.1053, 'col_2_error_rate': 0.3455, 'col_3_error_rate': 0.0, 'col_4_error_rate': 0.0, 'col_5_error_rate': 0.0952, 'col_6_error_rate': 0.0, 'col_7_error_rate': 0.069, 'total_error_rate': 0.1169, 'clf_aucroc': 0.8674}
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
{'nrmse': 0.0002, 'col_0_error_rate': 0.2424, 'col_1_error_rate': 0.075, 'col_2_error_rate': 0.1667, 'col_3_error_rate': 0.1471, 'col_4_error_rate': 0.0541, 'col_5_error_rate': 0.175, 'col_6_error_rate': 0.15, 'col_7_error_rate': 0.1212, 'total_error_rate': 0.1399, 'clf_aucroc': 0.8879}
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
{'nrmse': 0.0032, 'col_0_error_rate': 0.1957, 'col_1_error_rate': 0.0811, 'col_2_error_rate': 0.2

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
{'nrmse': 0.0044, 'col_0_error_rate': 0.3409, 'col_1_error_rate': 0.0741, 'col_2_error_rate': 0.2338, 'col_3_error_rate': 0.1317, 'col_4_error_rate': 0.0823, 'col_5_error_rate': 0.1813, 'col_6_error_rate': 0.1824, 'col_7_error_rate': 0.0955, 'total_error_rate': 0.1613, 'clf_aucroc': 0.8172}
nrmse 0.00478 20
col_0_error_rate 0.25558000000000003 20
col_1_error_rate 0.08854 20
col_2_error_rate 0.22468 20
col_3_error_rate 0.11988000000000001 20
col_4_error_rate 0.07731999999999999 20
col_5_error_rate 0.21338 20
col_6_error_rate 0.16416 20
col_7_error_rate 0.09744000000000001 20
total_error_rate 0.15496 20
clf_aucroc 0.8669 20


{5: {'nrmse': 0.0034200000000000003,
  'col_0_error_rate': 0.18082,
  'col_1_error_rate': 0.07186000000000001,
  'col_2_error_rate': 0.21538,
  'col_3_error_rate': 0.09068000000000001,
  'col_4_error_rate': 0.02232,
  'col_5_error_rate': 0.17712,
  'col_6_error_rate': 0.08834,
  'col_7_error_rate': 0.0721,
  'total_error_rate': 0.11785999999999999,
  'clf_aucroc': 0.86954},
 10: {'nrmse': 0.00346,
  'col_0_error_rate': 0.23264,
  'col_1_error_rate': 0.06536,
  'col_2_error_rate': 0.23500000000000001,
  'col_3_error_rate': 0.11954000000000001,
  'col_4_error_rate': 0.04464,
  'col_5_error_rate': 0.16388,
  'col_6_error_rate': 0.11488000000000001,
  'col_7_error_rate': 0.0871,
  'total_error_rate': 0.13208,
  'clf_aucroc': 0.86922},
 15: {'nrmse': 0.00438,
  'col_0_error_rate': 0.23842,
  'col_1_error_rate': 0.09246,
  'col_2_error_rate': 0.21888000000000002,
  'col_3_error_rate': 0.11667999999999998,
  'col_4_error_rate': 0.08203999999999999,
  'col_5_error_rate': 0.22566000000000003,
 